In [ ]:
#THIS IS THE FIRST ATTEMPT FOR THE BUSINESS PLAN PREDICTION MODEL

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, pipeline

# Data preprocessing
class BusinessPlanDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        plan = self.data[idx]
        input_ids = self.tokenizer.encode(plan, max_length=self.max_length, truncation=True, padding='max_length', return_tensors='pt')
        attention_mask = (input_ids != self.tokenizer.pad_token_id).float()
        return input_ids.squeeze(), attention_mask.squeeze()

# Model architecture
class BusinessPlanGenerator(nn.Module):
    def __init__(self, config, input_size):
        super().__init__()
        self.gpt2 = GPT2LMHeadModel(config)
        self.input_projection = nn.Linear(input_size, config.n_embd)

    def forward(self, input_ids, attention_mask, input_params):
        embedded_inputs = self.input_projection(input_params)
        outputs = self.gpt2(input_ids, attention_mask=attention_mask, inputs_embeds=embedded_inputs)
        return outputs.logits

# Training
def train(model, dataset, optimizer, device, num_epochs, save_path):
    model.train()
    for epoch in range(num_epochs):
        for batch in dataset:
            input_ids, attention_mask = batch
            input_ids, attention_mask = input_ids.to(device), attention_mask.to(device)
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask, input_params)
            loss = nn.CrossEntropyLoss()(outputs.view(-1, outputs.size(-1)), input_ids.view(-1))
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")
    torch.save(model.state_dict(), save_path)

# Generation
def generate_business_plan(model, input_params, device, tokenizer, max_length=1024, num_return_sequences=1):
    model.eval()
    input_ids = torch.zeros(1, 1, dtype=torch.long, device=device)
    attention_mask = torch.ones(1, 1, dtype=torch.long, device=device)
    input_params = torch.tensor([input_params], dtype=torch.float, device=device)

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        input_params=input_params,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        num_beams=5,
        early_stopping=True,
    )

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

# Main
if __:
    # Load dataset
    dataset = BusinessPlanDataset(data, tokenizer, max_length=1024)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

    # Load pre-trained LLM
    config = GPT2Config.from_pretrained('gpt2-medium')
    model = BusinessPlanGenerator(config, input_size=len(input_params)).to(device)
    model.gpt2.load_state_dict(torch.load('gpt2-medium.pth'))

    # Fine-tune the model
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    train(model, dataloader, optimizer, device, num_epochs=10, save_path='business-plan-generator.pth')

    # Generate a business plan
    input_params = [10, 15, 20, 1000, 500]
    business_plan = generate_business_plan(model, input_params, device, tokenizer)
    print(business_plan)

In [1]:
#USING CHATGPT PROMPTS

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from transformers import GPT2LMHeadModel, GPT2Tokenizer, GPT2Config, pipeline

# Data preprocessing
class BusinessPlanDataset(torch.utils.data.Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        crop_allocation, budget, land_area = self.data[idx]
        input_params = torch.tensor([crop_allocation, budget, land_area], dtype=torch.float)
        return input_params

# Model architecture
class BusinessPlanGenerator(nn.Module):
    def __init__(self, config, input_size):
        super().__init__()
        self.gpt2 = GPT2LMHeadModel(config)
        self.input_projection = nn.Linear(input_size, config.n_embd)

    def forward(self, input_ids, attention_mask, input_params):
        embedded_inputs = self.input_projection(input_params)
        outputs = self.gpt2(input_ids, attention_mask=attention_mask, inputs_embeds=embedded_inputs)
        return outputs.logits

# Training
def train(model, dataset, optimizer, device, num_epochs, save_path):
    model.train()
    for epoch in range(num_epochs):
        for batch in dataset:
            input_params = batch.to(device)
            optimizer.zero_grad()
            outputs = model(None, None, input_params)
            loss = nn.MSELoss()(outputs, input_params)
            loss.backward()
            optimizer.step()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")
    torch.save(model.state_dict(), save_path)

# Generation
def generate_business_plan(model, input_params, device, tokenizer, llm, max_length=1024, num_return_sequences=1):
    model.eval()
    input_params = torch.tensor([input_params], dtype=torch.float, device=device)
    outputs = model(None, None, input_params)
    
    prompt = f"Based on the crop allocation of {input_params[0].item()}, {input_params[1].item()}, and {input_params[2].item()} square meters, a budget of ${input_params[1].item()}, and a total land area of {input_params[2].item()} square meters, generate a detailed business plan."
    business_plan = llm(prompt, max_length=max_length, num_return_sequences=num_return_sequences, do_sample=True, top_k=50, top_p=0.95, num_beams=5, early_stopping=True)[0]['generated_text']
    
    return business_plan

# Main
if __:
    # Load dataset
    dataset = BusinessPlanDataset(data, tokenizer, max_length=1024)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

    # Load pre-trained LLM
    config = GPT2Config.from_pretrained('gpt2-medium')
    model = BusinessPlanGenerator(config, input_size=3).to(device)
    model.gpt2.load_state_dict(torch.load('gpt2-medium.pth'))

    # Fine-tune the model
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
    train(model, dataloader, optimizer, device, num_epochs=10, save_path='business-plan-generator.pth')

    # Generate a business plan
    input_params = [10, 15, 20, 1000, 500]
    llm = pipeline('text-generation', model='gpt2')
    business_plan = generate_business_plan(model, input_params, device, tokenizer, llm)
    print(business_plan)